# Model Context Protocol (MCP)
MCP는 유연한 LLM 어플리케이션 통합을 위한 프로토콜

## Client-Server Architecture
MCP는 Client-Server 구조를 따르고, 3가지 컴포넌트로 구성됨.
- Host : LLM 어플리케이션
- Clinet : Host속에 존재해서 Server와 1:1 연결을 맺는 구성요소
- Server : Context, Tools, Prompt를 Client에 제공


## Protocol layer
프로토콜 레이어는 통신 패턴을 다루는데 사용됨

## Transport layer
전송 계층은 실질적인 커뮤니케이션이 일어나는 파트로, JSON형식의 데이터를 교환함.
두가지 통신 매커니즘이 있음.
- Stdio : 로컬 환경에서 사용
- HTTP with SSE : 클라이언트가 서버에 POST를 하는 경우, 서버측에서 클라이언트에 메세지를 보내는 경우

## Connection Lifecycle 
1. Initialzation : 3-way 핸드쉐이크처럼 초기 통신 단계
2. Message Exchange : 메세지를 교환하는 부분으로 2가지 패턴이 주요함 
- Request-Response : 클라이언트-서버간 request/response 통신
- Notifications : 1방향 메세지
3. Termination : 연결 종료

In [1]:
import asyncio
import mcp.types as types
from mcp.server import Server
from mcp.server.stdio import stdio_server

app = Server("example-server")

@app.list_resources()
async def list_resources() -> list[types.Resource]:
    return [
        types.Resource(
            uri="example://resource",
            name="Example Resource"
        )
    ]

async def main():
    async with stdio_server() as streams:
        await app.run(
            streams[0],
            streams[1],
            app.create_initialization_options()
        )

if __name__ == "__main__":
    asyncio.run(main())

ModuleNotFoundError: No module named 'mcp'

## Resources
LLM에 사용될 데이터를 가져오는 개념. 
클라이언트가 리소스를 어떻게, 언제 사용할지 결정.

### 데이터의 종류
- 파일
- 데이터베이스
- API 
- 이미지 등

### Resource URI
리소스 파일은 다음 형식으로 구성
```
# format
[protocol]://[host]/[path]
# example
file:///home/user/documents/report.pdf
screen://localhost/display1
```

### Resource Types
- Text resources (UTF-8): source code , json file, plain text, and etc
- Binary resources (base64) : Image, PDF, Vedio, and etc   

### Resource Method
We can use two main methods to use data
- resources/list : direct access to resources
```
{
  uri: string;           // Unique identifier for the resource
  name: string;          // Human-readable name
  description?: string;  // Optional description
  mimeType?: string;     // Optional MIME type
}
```

- resources/read : The server responds with a list of resource contents.
```
{
  contents: [
    {
      uri: string;        // The URI of the resource
      mimeType?: string;  // Optional MIME type

      // One of:
      text?: string;      // For text resources
      blob?: string;      // For binary resources (base64 encoded)
    }
  ]
}
```

### Updating Resource
Two methos supprots real-time data update :
- notifications/resources/list_changed : Server can notify clients when list of resources are changed
- resources/subscribe : client can subsribe server resources by using with resource URI
- resources/read : client can fetch lastest resource
- resources/unsubscribe : client can unsubscribe

```
app = Server("example-server")

@app.list_resources()
async def list_resources() -> list[types.Resource]:
    return [
        types.Resource(
            uri="file:///logs/app.log",
            name="Application Logs",
            mimeType="text/plain"
        )
    ]

@app.read_resource()
async def read_resource(uri: AnyUrl) -> str:
    if str(uri) == "file:///logs/app.log":
        log_contents = await read_log_file()
        return log_contents

    raise ValueError("Resource not found")

# Start server
async with stdio_server() as streams:
    await app.run(
        streams[0],
        streams[1],
        app.create_initialization_options()
    )
```


# Prompt
재활용 가능한 프롬프트와 워크플로우를 만드는 컨셉
MCP에서의 프롬프트가 할 수 있는 행위:
- 다양한 인자를 사용
- 컨텍스트에 리소스 포함
- 여러 행위를 묶음
- 워크플로우에 대한 가이드
- UI

Endpoints
- prompts/list : 사용가능한 프롬프트 목록
- prompts/get : 사용할 프롬프트를 가져옴

In [ ]:
from mcp.server import Server
import mcp.types as types

# Define available prompts
PROMPTS = {
    "git-commit": types.Prompt(
        name="git-commit",
        description="Generate a Git commit message",
        arguments=[
            types.PromptArgument(
                name="changes",
                description="Git diff or description of changes",
                required=True
            )
        ],
    ),
    "explain-code": types.Prompt(
        name="explain-code",
        description="Explain how code works",
        arguments=[
            types.PromptArgument(
                name="code",
                description="Code to explain",
                required=True
            ),
            types.PromptArgument(
                name="language",
                description="Programming language",
                required=False
            )
        ],
    )
}

# Initialize server
app = Server("example-prompts-server")

@app.list_prompts()
async def list_prompts() -> list[types.Prompt]:
    return list(PROMPTS.values())

@app.get_prompt()
async def get_prompt(
    name: str, arguments: dict[str, str] | None = None
) -> types.GetPromptResult:
    if name not in PROMPTS:
        raise ValueError(f"Prompt not found: {name}")

    if name == "git-commit":
        changes = arguments.get("changes") if arguments else ""
        return types.GetPromptResult(
            messages=[
                types.PromptMessage(
                    role="user",
                    content=types.TextContent(
                        type="text",
                        text=f"Generate a concise but descriptive commit message "
                        f"for these changes:\n\n{changes}"
                    )
                )
            ]
        )

    if name == "explain-code":
        code = arguments.get("code") if arguments else ""
        language = arguments.get("language", "Unknown") if arguments else "Unknown"
        return types.GetPromptResult(
            messages=[
                types.PromptMessage(
                    role="user",
                    content=types.TextContent(
                        type="text",
                        text=f"Explain how this {language} code works:\n\n{code}"
                    )
                )
            ]
        )

    raise ValueError("Prompt implementation not found")

# Tools 
LLM이 할 행동을 구축하는 부분 : 외부 시스템과 상호작용을하고, 계산을 수행할 수 있음.

MCP에서 Tool은 서버가 실행가능한 함수를 실행할 수 있도록 지원함.
주요 엔드 포인트는 다음과 같음
- tools/list : 실행가능한 tool 목록
- tools/call : tool 실행

```
app = Server("example-server")

@app.list_tools()
async def list_tools() -> list[types.Tool]:
    return [
        types.Tool(
            name="calculate_sum",
            description="Add two numbers together",
            inputSchema={
                "type": "object",
                "properties": {
                    "a": {"type": "number"},
                    "b": {"type": "number"}
                },
                "required": ["a", "b"]
            }
        )
    ]

@app.call_tool()
async def call_tool(
    name: str,
    arguments: dict
) -> list[types.TextContent | types.ImageContent | types.EmbeddedResource]:
    if name == "calculate_sum":
        a = arguments["a"]
        b = arguments["b"]
        result = a + b
        return [types.TextContent(type="text", text=str(result))]
    raise ValueError(f"Tool not found: {name}")
```


# 샘플링
Sampling is a powerful MCP feature that allows servers to request LLM completions through the client, enabling sophisticated agentic behaviors while maintaining security and privacy.
샘플링은 LLM작업을 완료하는 파트 
Human-in-the-loop임

앤드포인트
- sampling/createMessage : 서버가 클라이언트에 보냄 -> 클라이언트가 리뷰 진행 & LLM 실행 -> 리뷰 결과를 서버에 전달

# Roots
서버가 수행할 범위를 지정 
URI - 서버 작업공간을 나타내는 URI -> 파일 시스템 처럼 동작 : 리소스가 어딨는지 쓰기 위해서, 다양한 리소스를 한번에 쓰기 위해서  

# Transport (전송)
클라이언트와 서버사이의 통신임, JSON-RPC 2.0을 기본으로 함. 
두가지 구현이 있음
- stdio (standart input/output) : command-line 도구를 사용할때, local에서 사용할 때, 간단한 구현할 때 사용
```
app = Server("example-server")

async with stdio_server() as streams:
    await app.run(
        streams[0],
        streams[1],
        app.create_initialization_options()
    )
```
```
params = StdioServerParameters(
    command="./server",
    args=["--option", "value"]
)

async with stdio_client(params) as streams:
    async with ClientSession(streams[0], streams[1]) as session:
        await session.initialize()
```
- sse (server-sent events) : 서버 -> 클라이언트 스트리밍이 필요할 때, 제한된 네트워크에서 사용할 때
```
from mcp.server.sse import SseServerTransport
from starlette.applications import Starlette
from starlette.routing import Route

app = Server("example-server")
sse = SseServerTransport("/messages")

async def handle_sse(scope, receive, send):
    async with sse.connect_sse(scope, receive, send) as streams:
        await app.run(streams[0], streams[1], app.create_initialization_options())

async def handle_messages(scope, receive, send):
    await sse.handle_post_message(scope, receive, send)

starlette_app = Starlette(
    routes=[
        Route("/sse", endpoint=handle_sse),
        Route("/messages", endpoint=handle_messages, methods=["POST"]),
    ]
)
```
```
async with sse_client("http://localhost:8000/sse") as streams:
    async with ClientSession(streams[0], streams[1]) as session:
        await session.initialize()
```

- custom 도 가능함